In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import re

from doc_metrics import csv_to_rows_of_strings, RowColumnView, Metrics

In [ ]:
# Traffic metrics for subsequent days, combined
day_a = r'readthedocs_traffic_analytics_jupyterlab_2023-12-05_2024-03-04.csv'
day_a_plus_one = r'readthedocs_traffic_analytics_jupyterlab_2023-12-06_2024-03-05.csv'

metrics_a = Metrics(csv_to_rows_of_strings(path=day_a))
metrics_b = Metrics(csv_to_rows_of_strings(path=day_a_plus_one))
metrics_c = Metrics.build(path=[day_a, day_a_plus_one])
allmet = [metrics_a, metrics_b, metrics_c]

In [ ]:
[m.total_views() for m in allmet]

In [ ]:
[m.most_popular_pages(3) for m in allmet]

In [ ]:
[m.most_popular_versions(3) for m in allmet]

In [ ]:
# Search metrics for subsequent days, merged
sday_a = r'readthedocs_search_analytics_jupyterlab_2023-12-05_2024-03-04.csv'
sday_a_plus_one = r'readthedocs_search_analytics_jupyterlab_2023-12-06_2024-03-05.csv'

smetrics_a = Metrics(csv_to_rows_of_strings(path=sday_a))
smetrics_b = Metrics(csv_to_rows_of_strings(path=sday_a_plus_one))
smetrics_c = Metrics.build(path=[sday_a, sday_a_plus_one])
sallmet = [smetrics_a, smetrics_b, smetrics_c]

In [ ]:
smetrics_c.most_popular_queries(10)

In [ ]:
[m.most_popular_queries(8) for m in sallmet]

In [ ]:
most_pop = metrics_c.most_popular_pages(25)
fig, ax = plt.subplots()  # Create a figure containing a single axes.
ax.invert_yaxis()
ax.barh([i[0] for i in most_pop], [i[1] for i in most_pop])  # Plot some data on the axes.